In [60]:
#program to extract image from pdf adn save in folder

import PyPDF2
import os
import sys
from PIL import Image
import io


#extract images from pdf without using fitz
def extract_images_from_pdf(pdf_path, output_path):
    pdf_document = PyPDF2.PdfFileReader(pdf_path)
    for current_page_number in range(pdf_document.getNumPages()):
        current_page = pdf_document.getPage(current_page_number)
        if "/XObject" in current_page["/Resources"]:
            x_object = current_page["/Resources"]["/XObject"].getObject()
            for obj in x_object:
                if x_object[obj]["/Subtype"] == "/Image":
                    image = x_object[obj]
                    image_bytes = image._data
                    image = Image.open(io.BytesIO(image_bytes))
                    image.save(os.path.join(output_path, f"{current_page_number}.png"))

if __name__ == "__main__":
    pdf_path = "123456.pdf"
    output_path = "./output"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    extract_images_from_pdf(pdf_path, output_path)
    print(f"Images extracted from {pdf_path} and saved in {output_path} folder")

Images extracted from 123456.pdf and saved in ./output folder


In [26]:
#run ocr on ever image in the output folder and print data between coordinates 668, 1112, 2740, 1390

import pytesseract
from PIL import Image
import os
import fitz


#no text is shown in the output

def find_word_in_pdf(pdf_path, word):
    pdf_document = fitz.open(pdf_path)
    for current_page_number in range(len(pdf_document)):
        page = pdf_document.load_page(current_page_number)
        text = page.get_text()
        if word.lower() in text.lower():
            print(f"Word '{word}' found on page {current_page_number + 1}")
    pdf_document.close()

if __name__ == "__main__":
    pdf_path = "123456.pdf"
    word = "COMPLAINT"
    find_word_in_pdf(pdf_path, word)

    

In [31]:
import matplotlib.pyplot as plt

In [58]:
#run ocr on ever image in the output folder and print data between coordinates 668, 1112, 2740, 1390 using easyocr

import easyocr
from PIL import Image
import os
import numpy as np

final={}

def ocr_on_images(images_folder_path):
    reader = easyocr.Reader(["en"])
    for image_name in os.listdir(images_folder_path):
        image_path = os.path.join(images_folder_path, image_name)
        image = Image.open(image_path)

        #crop the image to get the required coordinates x1=668, y1=1112, x2=2740, y2=1390
        image = image.crop((668, 1112, 2740, 1390))
        image = np.array(image)
        result = reader.readtext(image)
        
        # for detection in result:
        #     if detection[0][1] >= 668 and detection[0][3] <= 1112:
        #         print(f"Text between coordinates 668, 1112, 2740, 1390 is: {detection[1]}")
        if result==[]:
            print(image_name[0], "No text found")
            
        else:
            print(image_name[0], result[0][1])

        final[image_name[0]]="No text found" if result==[] else result[0][1]


    print(final)
        # sort the dictionary according to keys
    key_list=sorted(final.keys())
    sorted_dict = {}
    for key in key_list:
        sorted_dict[key] = final[key]

    print(sorted_dict)

    flg=0


    for key in sorted_dict:
        if sorted_dict[key]=="COMPLAINT":
            #store {key}.png in complaint folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./complaint/{key}.png")
            flg=1

        elif sorted_dict[key]=="SUMMONS":
            #store {key}.png in summons folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./summons/{key}.png")
            flg=2

        elif sorted_dict[key]=="Appearance By Attorney In Civil Case":
            #store {key}.png in attorney folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./appearance/{key}.png")
            flg=3

        else:
            if flg==1:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./complaint/{key}.png")

            elif flg==2:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./summons/{key}.png")

            elif flg==3:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./appearance/{key}.png")

    #convert images into pdf
    


    

if __name__ == "__main__":
    images_folder_path = "./output"
    ocr_on_images(images_folder_path)


4 Appearance By Attorney In Civil Case
2 No text found
3 SUMMONS
1 No text found
0 COMPLAINT
{'4': 'Appearance By Attorney In Civil Case', '2': 'No text found', '3': 'SUMMONS', '1': 'No text found', '0': 'COMPLAINT'}
{'0': 'COMPLAINT', '1': 'No text found', '2': 'No text found', '3': 'SUMMONS', '4': 'Appearance By Attorney In Civil Case'}


In [61]:
pip freeze | grep PyPDF2

PyPDF2==2.12.1
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.6/270.6 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: lazy-loader
    Found existing installation: lazy_loader 0.3
    Uninstalling lazy_loader-0.3:
      Successfully uninstalled lazy_loader-0.3

[notice] A new release of pip is available: 

In [92]:
import PyPDF2
import os
from PIL import Image
import io
import easyocr
import numpy as np
import configparser

def extract_images_from_pdf(pdf_path, output_path):
    pdf_document = PyPDF2.PdfFileReader(pdf_path)
    for current_page_number in range(pdf_document.getNumPages()):
        current_page = pdf_document.getPage(current_page_number)
        if "/XObject" in current_page["/Resources"]:
            x_object = current_page["/Resources"]["/XObject"].getObject()
            for obj in x_object:
                if x_object[obj]["/Subtype"] == "/Image":
                    image = x_object[obj]
                    image_bytes = image._data
                    image = Image.open(io.BytesIO(image_bytes))
                    image.save(os.path.join(output_path, f"{current_page_number}.png"))

def images_to_pdf(folder_path, output_pdf):
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    images = [Image.open(image) for image in image_paths]
    images[0].save(output_pdf, save_all=True, append_images=images[1:])

def ocr_on_images(images_folder_path, config):
    reader = easyocr.Reader(["en"])
    
    complaint_output = config['Directory']['COMPLAINT_OUTPUT']
    summons_output = config['Directory']['SUMMONS_OUTPUT']
    appearance_output = config['Directory']['APPEARANCE_OUTPUT']
    
    ocr_location = config['OCR']['Loc1'].split(',')
    x1, y1, x2, y2 = map(int, ocr_location)
    
    for image_name in os.listdir(images_folder_path):
        image_path = os.path.join(images_folder_path, image_name)
        image = Image.open(image_path)

        # Crop the image based on the OCR location provided in the configuration file
        image = image.crop((x1, y1, x2, y2))
        image = np.array(image)
        result = reader.readtext(image)

        if result == []:
            print(image_name[0], "No text found")
        else:
            print(image_name[0], result[0][1])

        final[image_name[0]] = "No text found" if result == [] else result[0][1]

    # Sort the dictionary according to keys
    key_list = sorted(final.keys())
    sorted_dict = {}
    for key in key_list:
        sorted_dict[key] = final[key]

    flg = 0

    for key in sorted_dict:
        if sorted_dict[key] == "COMPLAINT":
            # Store {key}.png in complaint folder
            image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
            image.save(os.path.join(complaint_output, f"{key}.png"))
            flg = 1

        elif sorted_dict[key] == "SUMMONS":
            # Store {key}.png in summons folder
            image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
            image.save(os.path.join(summons_output, f"{key}.png"))
            flg = 2

        elif sorted_dict[key] == "Appearance By Attorney In Civil Case":
            # Store {key}.png in appearance folder
            image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
            image.save(os.path.join(appearance_output, f"{key}.png"))
            flg = 3

        else:
            if flg == 1:
                image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
                image.save(os.path.join(complaint_output, f"{key}.png"))

            elif flg == 2:
                image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
                image.save(os.path.join(summons_output, f"{key}.png"))

            elif flg == 3:
                image = Image.open(os.path.join(images_folder_path, f"{key}.png"))
                image.save(os.path.join(appearance_output, f"{key}.png"))


if __name__ == "__main__":
    # Load the configuration file
    config = configparser.ConfigParser()
    config.read('config.cfg')  # Replace 'config.cfg' with the path to your configuration file
    
    # Extract input and output paths from the configuration file
    pdf_path = config['Directory']['INPUT']
    print(pdf_path)
    output_path = config['Directory']['OUTPUT']
    
    # Create the output directory if it doesn't exist
    # if not os.path.exists(output_path):
    #     os.makedirs(output_path)
    
    # Extract images from PDF
    extract_images_from_pdf(pdf_path, output_path)
    print(f"Images extracted from {pdf_path} and saved in {output_path} folder")
    
    # Perform OCR on images
    images_folder_path = output_path
    ocr_on_images(images_folder_path, config)

    # Convert separated images to PDFs for each category
    for folder, output_folder in [("complaint", config['Directory']['COMPLAINT_OUTPUT']), 
                                  ("summons", config['Directory']['SUMMONS_OUTPUT']), 
                                  ("appearance", config['Directory']['APPEARANCE_OUTPUT'])]:
        output_pdf = os.path.join(output_path, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_{folder}.pdf")
        images_to_pdf(output_folder, output_pdf)

/Users/maitreyakanitkar/Desktop/Coding&Misc/OCR_Pdf_Splitter/input/123456.pdf
Images extracted from /Users/maitreyakanitkar/Desktop/Coding&Misc/OCR_Pdf_Splitter/input/123456.pdf and saved in /Users/maitreyakanitkar/Desktop/Coding&Misc/OCR_Pdf_Splitter//output folder
4 Appearance By Attorney In Civil Case
2 No text found
3 SUMMONS
1 No text found
0 COMPLAINT
